In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from torch_geometric.nn import SAGEConv
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, max_error, median_absolute_error
from datetime import datetime, timedelta
from tqdm import tqdm
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.neighbors import BallTree
from math import radians
from statsmodels.tsa.stattools import acf
import matplotlib.dates as mdates
import warnings
from joblib import Parallel, delayed

class ImprovedSTRAP(nn.Module):
    """Improved Spatio-Temporal Real Estate Appraisal (STRAP) model with advanced sequence processing"""
    def __init__(self, input_dim, hidden_dim=128, num_gru_layers=2, dropout=0.1, num_neighborhoods=100):
        super(ImprovedSTRAP, self).__init__()
        
        # Property feature embedding
        self.property_embedding = nn.Linear(input_dim, hidden_dim)
        
        # Embedding layer for categorical features like neighborhood
        self.neighborhood_embedding = nn.Embedding(num_neighborhoods, 32)
        
        # Temporal GRU layers
        self.gru = nn.GRU(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_gru_layers,
            batch_first=True,
            dropout=dropout if num_gru_layers > 1 else 0
        )
        
        # 1D convolutional layer for local feature extraction
        self.conv1d = nn.Conv1d(
            in_channels=hidden_dim, 
            out_channels=hidden_dim, 
            kernel_size=3, 
            padding=1
        )
        
        # Transformer encoder for better sequence modeling
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=hidden_dim,
                nhead=4,
                dim_feedforward=hidden_dim*4,
                dropout=dropout,
                batch_first=True
            ),
            num_layers=2
        )
        
        # Spatial graph layers
        self.graph_conv = SAGEConv(hidden_dim, hidden_dim)
        
        # Normalization
        self.temporal_norm = nn.LayerNorm(hidden_dim)
        self.spatial_norm = nn.LayerNorm(hidden_dim)
        self.conv_norm = nn.LayerNorm(hidden_dim)
        self.transformer_norm = nn.LayerNorm(hidden_dim)
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.Tanh(),
            nn.Linear(64, 1),
            nn.Softmax(dim=1)
        )
        
        # Final prediction layers
        self.predictor = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1)
        )
        
        # Control flag for residual connections
        self.residual_connection = True
        
    def forward(self, x=None, edge_index=None, time_sequences=None, neighborhood_ids=None):
        """
        Forward pass for ImprovedSTRAP model
        
        Parameters:
        -----------
        x : Tensor, optional
            Regular property features
        edge_index : Tensor, optional
            Graph edge index for spatial connections
        time_sequences : Tensor, optional
            Temporal sequences for each property
        neighborhood_ids : Tensor, optional
            Neighborhood category IDs
            
        Returns:
        --------
        Tensor
            Predicted property prices
        """
        # Handle case where only time_sequences is provided
        if x is None and time_sequences is not None:
            # Get batch size from time_sequences instead
            batch_size = time_sequences.size(0)
            
            # Process temporal information using the sequences
            # Process with GRU
            gru_output, _ = self.gru(time_sequences)
            
            # Apply 1D convolution for local feature extraction
            # Transpose for conv1d [batch, seq_len, channels] -> [batch, channels, seq_len]
            conv_input = gru_output.transpose(1, 2)
            conv_output = self.conv1d(conv_input)
            # Transpose back [batch, channels, seq_len] -> [batch, seq_len, channels]
            conv_output = conv_output.transpose(1, 2)
            
            # Apply residual connection
            if self.residual_connection:
                conv_output = conv_output + gru_output
            
            # Apply transformer encoder for global context
            transformer_output = self.transformer_encoder(conv_output)
            
            # Apply residual connection
            if self.residual_connection:
                transformer_output = transformer_output + conv_output
            
            # Apply attention to focus on most relevant timesteps
            attention_weights = self.attention(transformer_output)
            context_vector = torch.sum(attention_weights * transformer_output, dim=1)
            temporal_output = self.temporal_norm(context_vector)
            
            # Since we don't have property features, use the temporal output for both parts
            # This is a modification to handle the sequence-only case
            spatial_output = temporal_output.clone()
            
            # Combine temporal and spatial outputs
            combined = torch.cat([temporal_output, spatial_output], dim=1)
            
            # Final prediction
            output = self.predictor(combined).squeeze(-1)
            
            return output
            
        # Original case where x is provided
        elif x is not None:
            batch_size = x.size(0)
            
            # Process property features
            property_emb = self.property_embedding(x)
            
            # Process neighborhood embeddings if provided
            if neighborhood_ids is not None:
                neighborhood_emb = self.neighborhood_embedding(neighborhood_ids)
                # Combine with property embedding
                property_emb = property_emb + neighborhood_emb
            
            # Process temporal information using sequences if provided
            if time_sequences is not None:
                # Process with GRU
                gru_output, _ = self.gru(time_sequences)
                
                # Apply 1D convolution for local feature extraction
                # Transpose for conv1d [batch, seq_len, channels] -> [batch, channels, seq_len]
                conv_input = gru_output.transpose(1, 2)
                conv_output = self.conv1d(conv_input)
                # Transpose back [batch, channels, seq_len] -> [batch, seq_len, channels]
                conv_output = conv_output.transpose(1, 2)
                
                # Apply residual connection
                if self.residual_connection:
                    conv_output = conv_output + gru_output
                
                # Apply transformer encoder for global context
                transformer_output = self.transformer_encoder(conv_output)
                
                # Apply residual connection
                if self.residual_connection:
                    transformer_output = transformer_output + conv_output
                
                # Apply attention to focus on most relevant timesteps
                attention_weights = self.attention(transformer_output)
                context_vector = torch.sum(attention_weights * transformer_output, dim=1)
                temporal_output = self.temporal_norm(context_vector)
            else:
                # Fallback for when sequences are not available
                # Create a sequence of property features (repeated) as before
                seq_length = 5  # Using a fixed sequence length
                sequence = property_emb.unsqueeze(1).repeat(1, seq_length, 1)
                
                # Process with GRU
                gru_output, _ = self.gru(sequence)
                
                # Apply attention to focus on most relevant timesteps
                attention_weights = self.attention(gru_output)
                context_vector = torch.sum(attention_weights * gru_output, dim=1)
                temporal_output = self.temporal_norm(context_vector)
            
            # Process with graph convolution if edge_index provided
            if edge_index is not None and edge_index.numel() > 0:
                spatial_output = self.graph_conv(property_emb, edge_index)
                spatial_output = self.spatial_norm(spatial_output)
            else:
                spatial_output = property_emb
            
            # Combine temporal and spatial outputs
            combined = torch.cat([temporal_output, spatial_output], dim=1)
            
            # Final prediction
            output = self.predictor(combined).squeeze(-1)
            
            return output
        
        else:
            # Neither x nor time_sequences provided
            raise ValueError("At least one of x or time_sequences must be provided")

class TemporalSequenceDataset(Dataset):
    """Dataset for temporal sequence data"""
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        # Convert to tensor with shape [sequence_length, features]
        sequence = torch.FloatTensor(self.sequences[idx])
        target = torch.FloatTensor([self.targets[idx]])
        
        return sequence, target

def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate the distance between two points using the Haversine formula."""
    R = 6371  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def create_spatial_features(df, k_neighbors=5, chunk_size=10000, n_jobs=3, train_data_only=None):
    """Memory-optimized spatial features for real estate predictions without listing_density
    
    Parameters:
    -----------
    df : DataFrame
        The data to create features for
    k_neighbors : int
        Number of neighbors to use for KNN features (reduced from original)
    chunk_size : int
        Size of chunks for parallel processing (increased for memory efficiency)
    n_jobs : int
        Number of parallel jobs (reduced to save memory)
    train_data_only : DataFrame, optional
        If provided, use only this data for computing the KNN features
        This prevents data leakage by ensuring test listings don't influence each other
    """
    # Use a smaller sample if dataset is very large
    reference_data = train_data_only if train_data_only is not None else df
    
    # If dataset is very large, sample it for reference data
    MAX_REFERENCE_SIZE = 10000000000  # Limit reference data size
    if len(reference_data) > MAX_REFERENCE_SIZE:
        print(f"Sampling {MAX_REFERENCE_SIZE} records from {len(reference_data)} for reference data to save memory")
        np.random.seed(42)
        reference_indices = np.random.choice(len(reference_data), MAX_REFERENCE_SIZE, replace=False)
        reference_data = reference_data.iloc[reference_indices].copy()
    
    # Adjust k_neighbors if the dataset is small
    actual_k = min(k_neighbors, len(reference_data) - 1)  # Make sure k is at most n-1
    if actual_k < k_neighbors:
        print(f"Reducing k_neighbors from {k_neighbors} to {actual_k} due to small sample size")
    
    # Paris city center (for distance calculations)
    city_center_lat, city_center_lon = 48.8566, 2.3522
    
    # Initialize spatial features (reduced set for memory efficiency)
    spatial_data = {
        'distance_to_center': [],
        'knn_price_mean': [],
        'poi_density': [],            # Simplified POI density
        'transport_access': [],       # Distance to nearest transit
        'amenity_score': []           # Neighborhood amenity quality
    }
    
    # Calculate basic distance features
    print("Calculating distance to center...")
    spatial_data['distance_to_center'] = df.apply(
        lambda row: calculate_distance(
            row['latitude'], 
            row['longitude'], 
            city_center_lat, 
            city_center_lon
        ),
        axis=1
    ).values
    
    # Enhanced BallTree processing with memory optimization
    print("Building spatial index tree...")
    all_coords = np.radians(reference_data[['latitude', 'longitude']].values)
    tree = BallTree(all_coords, metric='haversine')
    all_prices = reference_data['price'].values
    
    # Simulate POI density based on distance to center (memory efficient)
    print("Calculating POI density approximation...")
    spatial_data['poi_density'] = np.exp(-0.3 * spatial_data['distance_to_center'])
    
    # Simulate transport access (inverse of distance to center with some variations)
    print("Calculating transport access approximation...")
    np.random.seed(42)  # For reproducibility
    noise = np.random.normal(0, 0.3, size=len(df))  # Reduced noise to save computation
    spatial_data['transport_access'] = spatial_data['distance_to_center'] * 0.8 + noise
    spatial_data['transport_access'] = np.clip(spatial_data['transport_access'], 0.1, 10.0)
    
    # Simplified amenity score based on POI density and transport access
    print("Calculating simplified amenity score...")
    spatial_data['amenity_score'] = (
        0.6 * spatial_data['poi_density'] + 
        0.4 * (1 / (spatial_data['transport_access'] + 0.1))
    )
    spatial_data['amenity_score'] = np.clip(spatial_data['amenity_score'], 0.1, 1.0)
    
    # Process neighbors for price features in chunks
    print("Processing KNN price features in chunks...")
    knn_price_mean = np.zeros(len(df))
    
    for i in range(math.ceil(len(df) / chunk_size)):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, len(df))
        
        # Print status message only every 20 chunks
        if i % 20 == 0:
            print(f"Processing chunk {i+1}/{math.ceil(len(df) / chunk_size)}...")
        
        chunk_coords = np.radians(df.iloc[start_idx:end_idx][['latitude', 'longitude']].values)
        
        # Use smaller k for memory efficiency
        k_query = min(actual_k + 1, len(all_coords))
        distances, indices = tree.query(chunk_coords, k=k_query)
        
        # Process neighbor data
        if k_query > 1:
            # Get neighbor prices (excluding self)
            neighbor_prices = np.take(all_prices, indices[:, 1:])
            
            # Calculate mean price of neighbors
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)  # Ignore NaN warnings
                knn_price_mean[start_idx:end_idx] = np.nanmean(neighbor_prices, axis=1)
        
        # Free memory
        del distances, indices
        if 'neighbor_prices' in locals():
            del neighbor_prices
    
    spatial_data['knn_price_mean'] = knn_price_mean
    
    # Convert to DataFrame
    print("Creating spatial features DataFrame...")
    spatial_features = pd.DataFrame(spatial_data, index=df.index)
    
    # Standardize features
    print("Standardizing features...")
    features_to_standardize = list(spatial_data.keys())
    
    for col in features_to_standardize:
        mean_val = spatial_features[col].mean()
        std_val = spatial_features[col].std()
        # Avoid division by zero
        if std_val > 0:
            spatial_features[col] = (spatial_features[col] - mean_val) / std_val
        else:
            spatial_features[col] = 0  # Set to zero if std is zero
    
    return spatial_features

def create_calculated_features(df):
    """
    Create calculated features from existing data
    
    Parameters:
    -----------
    df : DataFrame
        The dataframe to add features to
    
    Returns:
    --------
    DataFrame
        Modified dataframe with new features
    """
    # Create a copy to avoid modifying the original
    df_copy = df.copy()
    
    # Bedroom ratio
    if 'bedrooms' in df_copy.columns and 'accommodates' in df_copy.columns:
        df_copy['bedroom_ratio'] = df_copy['bedrooms'] / df_copy['accommodates'].clip(lower=1)
    
    # Count amenities
    amenity_columns = df_copy.filter(like='has_').columns
    if len(amenity_columns) > 0:
        df_copy['amenity_count'] = df_copy[amenity_columns].sum(axis=1)
    
    # Luxury score
    luxury_amenities = ['has_hot_water', 'has_hair_dryer', 'has_dedicated_workspace']
    has_all_columns = all(col in df_copy.columns for col in luxury_amenities)
    
    if has_all_columns:
        df_copy['luxury_score'] = df_copy[luxury_amenities].sum(axis=1) / len(luxury_amenities)
    else:
        # If some columns are missing, use available ones or set to 0
        available_amenities = [col for col in luxury_amenities if col in df_copy.columns]
        if available_amenities:
            df_copy['luxury_score'] = df_copy[available_amenities].sum(axis=1) / len(available_amenities)
        else:
            df_copy['luxury_score'] = 0
    
    return df_copy

def create_temporal_sequences(df, window_size=5, batch_size=1000, n_jobs=-1):
    """
    Create time-based sequences for each listing in a memory-efficient and parallelized way.
    
    Parameters:
    -----------
    df : DataFrame
        The input data containing listing information with date and price
    window_size : int
        Number of consecutive time steps to include in each sequence
    batch_size : int
        Number of listings to process in each batch
    n_jobs : int
        Number of parallel jobs (-1 means using all available cores)
        
    Returns:
    --------
    dict : Dictionary containing sequences, targets, and listing IDs
    """
    print(f"Creating temporal sequences with window_size={window_size}...")
    
    # Make sure the DataFrame is sorted by date for each listing
    df = df.sort_values(['listing_id', 'date'])
    
    # Get unique listing IDs
    unique_listings = df['listing_id'].unique()
    total_listings = len(unique_listings)
    print(f"Processing sequences for {total_listings} unique listings")
    
    # Function to process a single listing
    def process_listing(listing_id):
        # Get data for this listing
        listing_data = df[df['listing_id'] == listing_id].sort_values('date')
        
        # Skip if we don't have enough data points
        if len(listing_data) < window_size:
            return [], [], []
        
        listing_sequences = []
        listing_targets = []
        listing_ids = []
        
        # Create sequences
        for i in range(len(listing_data) - window_size):
            # Input sequence (all features except listing_id, date, and price)
            seq_df = listing_data.iloc[i:i+window_size].copy()
            features = seq_df.drop(['listing_id', 'date', 'price'], axis=1, errors='ignore')
            
            # Target (next price)
            target = listing_data.iloc[i+window_size]['price']
            
            listing_sequences.append(features.values)
            listing_targets.append(target)
            listing_ids.append(listing_id)
        
        return listing_sequences, listing_targets, listing_ids
    
    # Process listings in batches with parallel execution
    all_sequences = []
    all_targets = []
    all_listing_ids = []
    
    # Define number of batches
    num_batches = (total_listings + batch_size - 1) // batch_size
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, total_listings)
        batch_listings = unique_listings[start_idx:end_idx]
        
        print(f"Processing batch {batch_idx+1}/{num_batches} ({len(batch_listings)} listings)")
        
        # Process batch in parallel
        batch_results = Parallel(n_jobs=n_jobs)(
            delayed(process_listing)(listing_id) for listing_id in batch_listings
        )
        
        # Unpack results
        for sequences, targets, ids in batch_results:
            if sequences:  # Only append if not empty
                all_sequences.extend(sequences)
                all_targets.extend(targets)
                all_listing_ids.extend(ids)
        
        # Force garbage collection to free memory
        import gc
        gc.collect()
    
    print(f"Created {len(all_sequences)} sequences from {total_listings} listings")
    
    # Convert lists to numpy arrays for better memory usage and performance
    result = {
        'sequences': np.array(all_sequences) if all_sequences else np.array([]),
        'targets': np.array(all_targets) if all_targets else np.array([]),
        'listing_ids': np.array(all_listing_ids) if all_listing_ids else np.array([])
    }
    
    # Print memory usage info
    sequences_memory = result['sequences'].nbytes / (1024 * 1024)
    print(f"Sequences memory usage: {sequences_memory:.2f} MB")
    
    return result

def train_improved_strap_epoch(model, train_loader, optimizer, criterion, device):
    """Train improved STRAP model for one epoch"""
    model.train()
    epoch_loss = 0
    batch_count = 0
    
    for batch in train_loader:
        # Handle different types of batches (sequences vs. regular features)
        if isinstance(batch[0], tuple) and len(batch[0]) == 2:
            # This is a sequence batch
            sequences, targets = batch
            sequences = sequences.to(device)
            targets = targets.to(device).squeeze(-1)
            
            # Forward pass with sequences
            outputs = model(x=None, time_sequences=sequences)
        else:
            # Regular batch
            batch_x, batch_y = batch
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            # Forward pass without sequences
            outputs = model(batch_x)
            targets = batch_y
        
        # Calculate loss
        loss = criterion(outputs, targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        epoch_loss += loss.item()
        batch_count += 1
    
    return epoch_loss / max(1, batch_count)

def evaluate_strap(model, X_test_tensor, y_test, device):
    """Evaluate STRAP model on test data"""
    model.eval()
    X_test_tensor = X_test_tensor.to(device)
    
    with torch.no_grad():
        y_pred = model(X_test_tensor).cpu().numpy()
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred) if len(set(y_test)) > 1 else np.nan
    with np.errstate(divide='ignore', invalid='ignore'):
        mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100
    
    return y_pred, {
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mape': mape
    }

def run_cv_improved_strap(train_path, train_ids_path, test_ids_path, output_dir=None, window_size=5, n_splits=5, sample_size=None):
    """
    Run improved STRAP neural network with sliding window cross-validation for predicting prices.
    Uses separate listing IDs for training and testing to prevent data leakage.
    Incorporates actual temporal sequences for each listing.
    
    Parameters:
    -----------
    train_path : str
        Path to the training CSV file
    train_ids_path : str
        Path to text file with training listing IDs
    test_ids_path : str
        Path to text file with test listing IDs
    output_dir : str, optional
        Directory to save results
    window_size : int, optional
        Size of the sliding window in days
    n_splits : int, optional
        Number of splits for time series cross-validation
    sample_size : int, optional
        Limit dataset to this number of random listings (for testing)
    """
    print(f"Processing dataset: {os.path.basename(train_path)}")
    
    # Load training data
    print("Loading data...")
    train_data = pd.read_csv(train_path)

    # Load listing IDs for train/test split
    print("Loading train/test listing IDs...")
    with open(train_ids_path, 'r') as f:
        train_listing_ids = [int(line.strip()) for line in f.readlines()]
        
    with open(test_ids_path, 'r') as f:
        test_listing_ids = [int(line.strip()) for line in f.readlines()]
    
    print(f"Loaded {len(train_listing_ids)} train IDs and {len(test_listing_ids)} test IDs")

    # Drop legacy price columns if they exist
    price_cols_to_remove = [
        'price_lag_1d', 'simulated_price',
    ]
    
    for col in price_cols_to_remove:
        if col in train_data.columns:
            print(f"Dropping {col} column from the dataset")
            train_data = train_data.drop(col, axis=1)
    
    # For testing - take only a small sample of listings if specified
    if sample_size:
        print(f"Limiting to {sample_size} random listings for testing")
        selected_train = np.random.choice(train_listing_ids, int(sample_size * 0.7), replace=False)
        selected_test = np.random.choice(test_listing_ids, int(sample_size * 0.3), replace=False)
        train_listing_ids = selected_train.tolist()
        test_listing_ids = selected_test.tolist()
    
    # Convert date column to datetime
    train_data['date'] = pd.to_datetime(train_data['date'])
    
    # Filter data to include only dates from 7/8/23 till 2/8/24
    start_date = pd.to_datetime('2023-07-08')
    end_date = pd.to_datetime('2024-02-08')
    train_data = train_data[(train_data['date'] >= start_date) & (train_data['date'] <= end_date)]
    
    # Create calculated features
    print("Creating calculated features...")
    train_data = create_calculated_features(train_data)
    
    # Check if we have data for the entire expected range
    print(f"Date range in filtered data: {train_data['date'].min()} to {train_data['date'].max()}")
    print(f"Number of days with data: {len(train_data['date'].dt.date.unique())}")

    # Get unique dates and ensure they are properly sorted
    unique_dates = sorted(train_data['date'].dt.date.unique())
    first_date = unique_dates[0]
    last_date = unique_dates[-1]

    # Check if there's a gap between the expected start date and actual first date
    if first_date > start_date.date():
        print(f"Warning: No data found between {start_date.date()} and {first_date}")
        print(f"Using available data starting from {first_date}")
    
    # Sort by date
    train_data = train_data.sort_values('date')
    
    # Extract neighborhood information for embedding
    if 'neighborhood' in train_data.columns:
        # Create mapping of neighborhood to integer IDs
        neighborhoods = sorted(train_data['neighborhood'].unique())
        neighborhood_mapping = {neighborhood: i for i, neighborhood in enumerate(neighborhoods)}
        
        # Add neighborhood_id column
        train_data['neighborhood_id'] = train_data['neighborhood'].map(neighborhood_mapping)
        num_neighborhoods = len(neighborhoods)
        print(f"Found {num_neighborhoods} unique neighborhoods")
    else:
        # If no neighborhood column, use a dummy value
        train_data['neighborhood_id'] = 0
        num_neighborhoods = 1
        print("No neighborhood column found, using dummy values")
    
    # Pre-compute spatial features for all data at once (before CV splits)
    print("Pre-computing spatial features for all data...")
    full_train_data = train_data[train_data['listing_id'].isin(train_listing_ids)]
    all_spatial_features = create_spatial_features(train_data, train_data_only=full_train_data)
    
    # Cache to disk if output directory is provided
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        cache_file = os.path.join(output_dir, 'cached_spatial_features.csv')
        all_spatial_features.to_csv(cache_file)
        print(f"Cached spatial features saved to {cache_file}")
    
    # Create explicit test periods - last 5 weeks (35 days) split into 5 equal parts (7 days each)
    last_35_days = unique_dates[-35:]
    
    # Define explicit test periods - each 7 days
    test_periods = []
    for i in range(n_splits):
        start_idx = i * 7
        end_idx = start_idx + 7
        # Make sure we don't go beyond the available data
        if end_idx <= len(last_35_days):
            test_periods.append((last_35_days[start_idx], last_35_days[end_idx-1]))
    
    # Adjust n_splits if we couldn't create enough test periods
    n_splits = len(test_periods)
    
    print(f"Created {n_splits} test periods:")
    for i, (test_start, test_end) in enumerate(test_periods):
        print(f"  Period {i+1}: {test_start} to {test_end}")
    
    # Initialize device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Enable cuDNN benchmark mode for optimized performance
    torch.backends.cudnn.benchmark = True
    
    # Storage for results
    cv_results = []
    feature_importance_over_time = []  # We'll simulate this for the NN model
    
    # Run time series cross-validation using our explicit test periods
    for i, (test_start, test_end) in enumerate(test_periods):
        print(f"\nSplit {i+1}/{n_splits}")
        
        # Define training period: everything before test_start
        train_end = pd.to_datetime(test_start) - pd.Timedelta(days=1)
        train_end_date = train_end.date()
        
        print(f"Training period: {unique_dates[0]} to {train_end_date}")
        print(f"Testing period: {test_start} to {test_end}")
        
        # Split by date first
        train_date_mask = train_data['date'].dt.date <= train_end_date
        test_date_mask = (train_data['date'].dt.date >= test_start) & (train_data['date'].dt.date <= test_end)
        
        date_filtered_train = train_data[train_date_mask]
        date_filtered_test = train_data[test_date_mask]
        
        # Now further split by listing IDs
        train_id_mask = date_filtered_train['listing_id'].isin(train_listing_ids)
        test_id_mask = date_filtered_test['listing_id'].isin(test_listing_ids)
        
        split_train_data = date_filtered_train[train_id_mask]
        split_test_data = date_filtered_test[test_id_mask]
        
        print(f"Train data: {len(split_train_data)} rows, {len(split_train_data['listing_id'].unique())} unique listings")
        print(f"Test data: {len(split_test_data)} rows, {len(split_test_data['listing_id'].unique())} unique listings")
        
        # Use cached spatial features instead of recomputing
        train_spatial = all_spatial_features.loc[split_train_data.index]
        test_spatial = all_spatial_features.loc[split_test_data.index]
        
        # Create temporal sequences from training data
        print("Creating temporal sequences from historical data...")
        train_sequences = create_temporal_sequences(
            df=split_train_data, 
            window_size=window_size,
            batch_size=500,  # Process 500 listings at a time
            n_jobs=-1        # Use all available CPU cores
        )
        
        # Prepare feature matrices for standard training
        X_train = pd.concat([
            split_train_data.drop(['listing_id', 'date', 'price'], axis=1), 
            train_spatial
        ], axis=1)
        
        X_test = pd.concat([
            split_test_data.drop(['listing_id', 'date', 'price'], axis=1), 
            test_spatial
        ], axis=1)
        
        y_train = split_train_data['price']
        y_test = split_test_data['price']
        
        # Extract neighborhood IDs
        neighborhood_ids_train = split_train_data['neighborhood_id'].values
        neighborhood_ids_test = split_test_data['neighborhood_id'].values
        
        # Ensure X_train and X_test have the same columns
        missing_cols = set(X_train.columns) - set(X_test.columns)
        for col in missing_cols:
            X_test[col] = 0
            
        # Ensure the columns are in the same order
        X_test = X_test[X_train.columns]
        
        # Create temporal dataset
        if len(train_sequences['sequences']) > 0:
            temporal_dataset = TemporalSequenceDataset(
                train_sequences['sequences'],
                train_sequences['targets']
            )
            
            temporal_loader = DataLoader(
                temporal_dataset,
                batch_size=64,
                shuffle=True,
                drop_last=False,
                num_workers=4,
                pin_memory=True
            )
            
            print(f"Created temporal dataset with {len(temporal_dataset)} sequences")
        else:
            temporal_loader = None
            print("No temporal sequences could be created - using standard features only")
        
        # Convert to PyTorch tensors
        X_train_tensor = torch.FloatTensor(X_train.values)
        y_train_tensor = torch.FloatTensor(y_train.values)
        X_test_tensor = torch.FloatTensor(X_test.values)
        neighborhood_ids_train_tensor = torch.LongTensor(neighborhood_ids_train)
        neighborhood_ids_test_tensor = torch.LongTensor(neighborhood_ids_test)
        
        # Initialize improved model for this split
        model = ImprovedSTRAP(
            input_dim=X_train.shape[1],
            hidden_dim=128,
            num_gru_layers=2,
            dropout=0.2,
            num_neighborhoods=num_neighborhoods
        ).to(device)
        
        # Define optimizer and loss function
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
        criterion = nn.MSELoss()
        
        # Create DataLoader for standard training
        standard_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
        standard_loader = torch.utils.data.DataLoader(
            standard_dataset, 
            batch_size=512,
            shuffle=True,
            drop_last=False,
            num_workers=4,
            pin_memory=True
        )
        
        # Train the model with early stopping
        print(f"Training on {len(X_train)} samples, testing on {len(X_test)} samples")
        
        num_epochs = 50
        patience = 7
        min_delta = 0.001
        
        best_val_loss = float('inf')
        best_model_state = None
        counter = 0
        
        # Create a validation set from training data
        train_size = int(0.8 * len(X_train_tensor))
        val_size = len(X_train_tensor) - train_size
        
        train_subset, val_subset = torch.utils.data.random_split(
            standard_dataset, 
            [train_size, val_size]
        )
        
        train_subset_loader = torch.utils.data.DataLoader(
            train_subset, 
            batch_size=1024,
            shuffle=True,
            drop_last=False,
            num_workers=4,
            pin_memory=True
        )
        
        val_subset_loader = torch.utils.data.DataLoader(
            val_subset, 
            batch_size=1024,
            shuffle=False,
            drop_last=False,
            num_workers=4,
            pin_memory=True
        )
        
        # Training loop with early stopping
        for epoch in range(num_epochs):
            # Train on standard features first
            model.train()
            train_loss = 0
            batch_count = 0
            
            for batch_x, batch_y in train_subset_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                
                # Forward pass with standard features
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                
                train_loss += loss.item()
                batch_count += 1
            
            # Now train on temporal sequences if available
            if temporal_loader is not None:
                for sequences, targets in temporal_loader:
                    sequences = sequences.to(device)
                    targets = targets.to(device).squeeze(-1)
                    
                    # Forward pass with temporal sequences
                    outputs = model(x=None, time_sequences=sequences)
                    loss = criterion(outputs, targets)
                    
                    # Backward and optimize
                    optimizer.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    
                    train_loss += loss.item()
                    batch_count += 1
            
            if batch_count > 0:
                train_loss /= batch_count
            
            # Validate on standard validation set
            model.eval()
            val_loss = 0
            val_count = 0
            
            with torch.no_grad():
                for val_x, val_y in val_subset_loader:
                    val_x = val_x.to(device)
                    val_y = val_y.to(device)
                    val_outputs = model(val_x)
                    batch_loss = criterion(val_outputs, val_y)
                    val_loss += batch_loss.item()
                    val_count += 1
            
            if val_count > 0:
                val_loss /= val_count
            
            # Print progress every 5 epochs
            if (epoch + 1) % 5 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
            
            # Check for improvement
            if val_loss < best_val_loss - min_delta:
                best_val_loss = val_loss
                best_model_state = model.state_dict().copy()
                counter = 0
            else:
                counter += 1
            
            # Early stopping
            if counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        # Load best model if we found one
        if best_model_state is not None:
            model.load_state_dict(best_model_state)
        
        # Final training on all training data
        print("Final training on all training data...")
        for _ in range(5):  # A few more epochs on all data
            # Train on standard features
            for batch_x, batch_y in standard_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                
                # Forward pass
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
            
            # Train on temporal sequences if available
            if temporal_loader is not None:
                for sequences, targets in temporal_loader:
                    sequences = sequences.to(device)
                    targets = targets.to(device).squeeze(-1)
                    
                    # Forward pass with temporal sequences
                    outputs = model(x=None, time_sequences=sequences)
                    loss = criterion(outputs, targets)
                    
                    # Backward and optimize
                    optimizer.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
        
        # Evaluate on test data
        model.eval()
        X_test_tensor = X_test_tensor.to(device)
        neighborhood_ids_test_tensor = neighborhood_ids_test_tensor.to(device)
        
        with torch.no_grad():
            y_pred = model(
                X_test_tensor, 
                neighborhood_ids=neighborhood_ids_test_tensor
            ).cpu().numpy()
        
        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred) if len(set(y_test)) > 1 else np.nan
        with np.errstate(divide='ignore', invalid='ignore'):
            mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100
        
        print(f"Split {i+1} Results - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")
        
        # Generate simulated feature importance for visualization
        feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': np.random.uniform(0, 1, size=X_train.shape[1]),  # Random placeholder values
            'split': i
        })
        feature_importance_over_time.append(feature_importance)
        
        # Store results for this split
        split_results = pd.DataFrame({
            'split': i,
            'date': split_test_data['date'],
            'listing_id': split_test_data['listing_id'],
            'price': y_test,
            'predicted': y_pred,
            'error': y_test - y_pred,
            'abs_error': np.abs(y_test - y_pred),
            'pct_error': np.abs((y_test - y_pred) / (y_test + 1e-8)) * 100
        })
        
        cv_results.append(split_results)
    
    # Combine all results
    all_results = pd.concat(cv_results, ignore_index=True)
    all_feature_importance = pd.concat(feature_importance_over_time, ignore_index=True)
    
    # Calculate overall metrics
    y_true = all_results['price']
    y_pred = all_results['predicted']
    
    overall_metrics = {
        'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
        'mae': mean_absolute_error(y_true, y_pred),
        'r2': r2_score(y_true, y_pred),
        'mape': np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100,
        'explained_variance': explained_variance_score(y_true, y_pred),
        'max_error': max_error(y_true, y_pred),
        'median_absolute_error': median_absolute_error(y_true, y_pred)
    }
    
    # Calculate split-level metrics
    split_metrics = []
    for split in range(n_splits):
        split_data = all_results[all_results['split'] == split]
        if not split_data.empty:
            y_true_split = split_data['price']
            y_pred_split = split_data['predicted']
            
            split_metrics.append({
                'split': split,
                'rmse': np.sqrt(mean_squared_error(y_true_split, y_pred_split)),
                'mae': mean_absolute_error(y_true_split, y_pred_split),
                'r2': r2_score(y_true_split, y_pred_split) if len(set(y_true_split)) > 1 else np.nan,
                'mape': np.mean(np.abs((y_true_split - y_pred_split) / (y_true_split + 1e-8))) * 100,
                'n_samples': len(y_true_split)
            })
    
    split_metrics_df = pd.DataFrame(split_metrics)
    
    # Calculate daily metrics
    all_results['date_str'] = pd.to_datetime(all_results['date']).dt.strftime('%Y-%m-%d')
    
    daily_metrics = []
    for day, group in all_results.groupby('date_str'):
        y_true_day = group['price']
        y_pred_day = group['predicted']
        
        daily_metrics.append({
            'date': day,
            'rmse': np.sqrt(mean_squared_error(y_true_day, y_pred_day)),
            'mae': mean_absolute_error(y_true_day, y_pred_day),
            'r2': r2_score(y_true_day, y_pred_day) if len(set(y_true_day)) > 1 else np.nan,
            'mape': np.mean(np.abs((y_true_day - y_pred_day) / (y_true_day + 1e-8))) * 100,
            'n_samples': len(y_true_day)
        })
    
    daily_metrics_df = pd.DataFrame(daily_metrics)
    daily_metrics_df['date'] = pd.to_datetime(daily_metrics_df['date'])
    daily_metrics_df = daily_metrics_df.sort_values('date')
    
    # Error analysis
    errors = all_results['error'].values
    error_autocorr = acf(errors, nlags=7)[1:]  # Exclude lag 0
    
    # Create a results dictionary
    evaluation_results = {
        'overall_metrics': overall_metrics,
        'split_metrics': split_metrics_df,
        'daily_metrics': daily_metrics_df,
        'all_results': all_results,
        'feature_importance': all_feature_importance,
        'error_autocorrelation': error_autocorr,
        'train_listings': len(train_listing_ids),
        'test_listings': len(test_listing_ids)
    }
    
    print("\n===== Running Permutation Test for Data Leakage =====")
    permutation_test_results = []
    
    for i, (test_start, test_end) in enumerate(test_periods):
        print(f"\nRunning permutation test for Split {i+1}/{n_splits}")
        
        # Get the test data for this split
        split_test_data = all_results[all_results['split'] == i]
        
        if len(split_test_data) == 0:
            continue
            
        # Create a copy with shuffled prices
        np.random.seed(42 + i)  # Different seed for each split
        permutation_test_df = split_test_data.copy()
        
        # Get original listing IDs and true prices
        listing_ids = permutation_test_df['listing_id'].values
        original_prices = permutation_test_df['price'].values
        
        # Shuffle prices within the test set
        shuffled_prices = original_prices.copy()
        np.random.shuffle(shuffled_prices)
        permutation_test_df['shuffled_price'] = shuffled_prices
        
        # Get model predictions (these remain the same)
        predictions = permutation_test_df['predicted'].values
        
        # Calculate metrics with shuffled data
        shuffled_rmse = np.sqrt(mean_squared_error(shuffled_prices, predictions))
        shuffled_mae = mean_absolute_error(shuffled_prices, predictions)
        shuffled_r2 = r2_score(shuffled_prices, predictions) if len(set(shuffled_prices)) > 1 else np.nan
        
        # Store results
        permutation_test_results.append({
            'split': i,
            'original_rmse': np.sqrt(mean_squared_error(original_prices, predictions)),
            'original_mae': mean_absolute_error(original_prices, predictions),
            'original_r2': r2_score(original_prices, predictions) if len(set(original_prices)) > 1 else np.nan,
            'shuffled_rmse': shuffled_rmse,
            'shuffled_mae': shuffled_mae,
            'shuffled_r2': shuffled_r2,
            'n_samples': len(permutation_test_df)
        })
        
        print(f"Split {i+1} Original vs Shuffled: R² {permutation_test_results[-1]['original_r2']:.4f} vs {shuffled_r2:.4f}")
        if shuffled_r2 > 0.3:
            print("WARNING: High R² (>0.3) on shuffled data indicates potential data leakage!")
    
    # Convert to DataFrame for easier analysis
    permutation_df = pd.DataFrame(permutation_test_results)
    
    # Calculate overall shuffled metrics
    all_original_prices = all_results['price'].values
    all_predictions = all_results['predicted'].values
    
    # Shuffle all prices together
    np.random.seed(42)
    all_shuffled_prices = all_original_prices.copy()
    np.random.shuffle(all_shuffled_prices)
    
    overall_shuffled_metrics = {
        'rmse': np.sqrt(mean_squared_error(all_shuffled_prices, all_predictions)),
        'mae': mean_absolute_error(all_shuffled_prices, all_predictions),
        'r2': r2_score(all_shuffled_prices, all_predictions),
        'mape': np.mean(np.abs((all_shuffled_prices - all_predictions) / (all_shuffled_prices + 1e-8))) * 100
    }
    
    print("\n===== Overall Permutation Test Results =====")
    print(f"Original R²: {overall_metrics['r2']:.4f}")
    print(f"Shuffled R²: {overall_shuffled_metrics['r2']:.4f}")
    
    if overall_shuffled_metrics['r2'] > 0.3:
        print("\nWARNING: High R² (>0.3) on shuffled data indicates serious data leakage!")
        print("This suggests your model is using information that shouldn't be available.")
    else:
        print("\nNo significant data leakage detected in the permutation test.")
    
    # Add permutation test results to evaluation results
    evaluation_results['permutation_test'] = permutation_df
    evaluation_results['overall_shuffled_metrics'] = overall_shuffled_metrics
    
    # Create permutation test plot
    plt.figure(figsize=(10, 6))
    
    x = np.arange(len(permutation_df))
    width = 0.35
    
    plt.bar(x - width/2, permutation_df['original_r2'], width, label='Original R²')
    plt.bar(x + width/2, permutation_df['shuffled_r2'], width, label='Shuffled R²')
    
    plt.axhline(y=0.3, color='r', linestyle='--', label='Leakage Threshold (R²=0.3)')
    plt.xlabel('CV Split')
    plt.ylabel('R²')
    plt.title('Permutation Test Results: Original vs. Shuffled Prices')
    plt.xticks(x, permutation_df['split'])
    plt.legend()
    plt.tight_layout()
    
    if output_dir:
        plt.savefig(os.path.join(output_dir, 'permutation_test.png'))
        
        # Save permutation test results
        permutation_file = os.path.join(output_dir, 'permutation_test_results.csv')
        permutation_df.to_csv(permutation_file, index=False)
        print(f"Permutation test results saved to {permutation_file}")
        
        # Update summary file
        with open(os.path.join(output_dir, 'summary.txt'), 'a') as f:
            f.write("\n\nPermutation Test Results:\n")
            f.write(f"Original R²: {overall_metrics['r2']:.6f}\n")
            f.write(f"Shuffled R²: {overall_shuffled_metrics['r2']:.6f}\n")
            
            if overall_shuffled_metrics['r2'] > 0.3:
                f.write("WARNING: Potential data leakage detected!\n")
    
    plt.show()

    # Save results if output directory is provided
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        
        # Save all results
        results_file = os.path.join(output_dir, 'cv_improved_strap_results.csv')
        all_results.to_csv(results_file, index=False)
        print(f"Results saved to {results_file}")
        
        # Save metrics
        metrics_file = os.path.join(output_dir, 'cv_improved_strap_metrics.csv')
        daily_metrics_df.to_csv(metrics_file, index=False)
        print(f"Daily metrics saved to {metrics_file}")
        
        # Save feature importance
        importance_file = os.path.join(output_dir, 'cv_improved_strap_feature_importance.csv')
        all_feature_importance.to_csv(importance_file, index=False)
        print(f"Feature importance saved to {importance_file}")
        
        # Save model parameters
        model_info = {
            'architecture': 'Improved STRAP (Spatio-Temporal Real estate APpraisal)',
            'input_dim': X_train.shape[1],
            'hidden_dim': 128,
            'num_gru_layers': 2,
            'dropout': 0.2,
            'num_neighborhoods': num_neighborhoods,
            'transformer_layers': 2,
            'transformer_heads': 4,
            'residual_connections': True
        }
        
        # Save summary
        with open(os.path.join(output_dir, 'summary.txt'), 'w') as f:
            f.write(f"Cross-Validation Improved STRAP Neural Network with Listing ID Split\n")
            f.write(f"==================================================================\n\n")
            f.write(f"Window size: {window_size} days\n")
            f.write(f"Number of splits: {n_splits}\n")
            f.write(f"Training period: {unique_dates[0]} to {unique_dates[-1]}\n")
            f.write(f"Number of training listings: {len(train_listing_ids)}\n")
            f.write(f"Number of test listings: {len(test_listing_ids)}\n\n")
            
            f.write(f"Model Architecture:\n")
            for k, v in model_info.items():
                f.write(f"  {k}: {v}\n")
            
            f.write(f"\nOverall Metrics:\n")
            for k, v in overall_metrics.items():
                f.write(f"  {k}: {v:.6f}\n")
    
    # Print summary
    print_cv_summary(evaluation_results)
    
    # Create plots
    plot_cv_results(evaluation_results)
    
    return evaluation_results

def run_cv_strap(train_path, train_ids_path, test_ids_path, output_dir=None, window_size=35, n_splits=5, sample_size=None):
    """
    Run STRAP neural network with sliding window cross-validation for predicting prices.
    Uses separate listing IDs for training and testing to prevent data leakage.
    
    Parameters:
    -----------
    train_path : str
        Path to the training CSV file
    train_ids_path : str
        Path to text file with training listing IDs
    test_ids_path : str
        Path to text file with test listing IDs
    output_dir : str, optional
        Directory to save results
    window_size : int, optional
        Size of the sliding window in days
    n_splits : int, optional
        Number of splits for time series cross-validation
    sample_size : int, optional
        Limit dataset to this number of random listings (for testing)
    """
    print(f"Processing dataset: {os.path.basename(train_path)}")
    
    # Load training data
    print("Loading data...")
    train_data = pd.read_csv(train_path)

    # Load listing IDs for train/test split
    print("Loading train/test listing IDs...")
    with open(train_ids_path, 'r') as f:
        train_listing_ids = [int(line.strip()) for line in f.readlines()]
        
    with open(test_ids_path, 'r') as f:
        test_listing_ids = [int(line.strip()) for line in f.readlines()]
    
    print(f"Loaded {len(train_listing_ids)} train IDs and {len(test_listing_ids)} test IDs")

    # Drop legacy price columns if they exist
    price_cols_to_remove = [
        'price_lag_1d', 'simulated_price',
        # 'price_lag_7d', 'price_lag_14d', 'price_lag_30d'
        # ,'rolling_mean_7d', 'rolling_max_7d', 'rolling_min_7d',
        # 'rolling_mean_14d', 'rolling_max_14d', 'rolling_min_14d',
        # 'rolling_mean_30d', 'rolling_max_30d', 'rolling_min_30d'
    ]
    
    for col in price_cols_to_remove:
        if col in train_data.columns:
            print(f"Dropping {col} column from the dataset")
            train_data = train_data.drop(col, axis=1)
    
    # For testing - take only a small sample of listings if specified
    if sample_size:
        print(f"Limiting to {sample_size} random listings for testing")
        selected_train = np.random.choice(train_listing_ids, int(sample_size * 0.7), replace=False)
        selected_test = np.random.choice(test_listing_ids, int(sample_size * 0.3), replace=False)
        train_listing_ids = selected_train.tolist()
        test_listing_ids = selected_test.tolist()
    
    # Convert date column to datetime
    train_data['date'] = pd.to_datetime(train_data['date'])
    
    # Filter data to include only dates from 7/8/23 till 2/8/24
    start_date = pd.to_datetime('2023-07-08')
    end_date = pd.to_datetime('2024-02-08')
    train_data = train_data[(train_data['date'] >= start_date) & (train_data['date'] <= end_date)]
    
    # Create calculated features
    print("Creating calculated features...")
    train_data = create_calculated_features(train_data)
    
    # Check if we have data for the entire expected range
    print(f"Date range in filtered data: {train_data['date'].min()} to {train_data['date'].max()}")
    print(f"Number of days with data: {len(train_data['date'].dt.date.unique())}")

    # Get unique dates and ensure they are properly sorted
    unique_dates = sorted(train_data['date'].dt.date.unique())
    first_date = unique_dates[0]
    last_date = unique_dates[-1]

    # Check if there's a gap between the expected start date and actual first date
    if first_date > start_date.date():
        print(f"Warning: No data found between {start_date.date()} and {first_date}")
        print(f"Using available data starting from {first_date}")
    
    # Sort by date
    train_data = train_data.sort_values('date')
    
    # Pre-compute spatial features for all data at once (before CV splits)
    print("Pre-computing spatial features for all data...")
    full_train_data = train_data[train_data['listing_id'].isin(train_listing_ids)]
    all_spatial_features = create_spatial_features(train_data, train_data_only=full_train_data)
    
    # Cache to disk if output directory is provided
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        cache_file = os.path.join(output_dir, 'cached_spatial_features.csv')
        all_spatial_features.to_csv(cache_file)
        print(f"Cached spatial features saved to {cache_file}")
    
    # Create explicit test periods - last 5 weeks (35 days) split into 5 equal parts (7 days each)
    last_35_days = unique_dates[-35:]
    
    # Define explicit test periods - each 7 days
    test_periods = []
    for i in range(n_splits):
        start_idx = i * 7
        end_idx = start_idx + 7
        # Make sure we don't go beyond the available data
        if end_idx <= len(last_35_days):
            test_periods.append((last_35_days[start_idx], last_35_days[end_idx-1]))
    
    # Adjust n_splits if we couldn't create enough test periods
    n_splits = len(test_periods)
    
    print(f"Created {n_splits} test periods:")
    for i, (test_start, test_end) in enumerate(test_periods):
        print(f"  Period {i+1}: {test_start} to {test_end}")
    
    # Initialize device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Enable cuDNN benchmark mode for optimized performance
    torch.backends.cudnn.benchmark = True
    
    # Storage for results
    cv_results = []
    feature_importance_over_time = []  # We'll simulate this for the NN model
    
    # Run time series cross-validation using our explicit test periods
    for i, (test_start, test_end) in enumerate(test_periods):
        print(f"\nSplit {i+1}/{n_splits}")
        
        # Define training period: everything before test_start
        train_end = pd.to_datetime(test_start) - pd.Timedelta(days=1)
        train_end_date = train_end.date()
        
        print(f"Training period: {unique_dates[0]} to {train_end_date}")
        print(f"Testing period: {test_start} to {test_end}")
        
        # Split by date first
        train_date_mask = train_data['date'].dt.date <= train_end_date
        test_date_mask = (train_data['date'].dt.date >= test_start) & (train_data['date'].dt.date <= test_end)
        
        date_filtered_train = train_data[train_date_mask]
        date_filtered_test = train_data[test_date_mask]
        
        # Now further split by listing IDs
        train_id_mask = date_filtered_train['listing_id'].isin(train_listing_ids)
        test_id_mask = date_filtered_test['listing_id'].isin(test_listing_ids)
        
        split_train_data = date_filtered_train[train_id_mask]
        split_test_data = date_filtered_test[test_id_mask]
        
        print(f"Train data: {len(split_train_data)} rows, {len(split_train_data['listing_id'].unique())} unique listings")
        print(f"Test data: {len(split_test_data)} rows, {len(split_test_data['listing_id'].unique())} unique listings")
        
        # Use cached spatial features instead of recomputing
        train_spatial = all_spatial_features.loc[split_train_data.index]
        test_spatial = all_spatial_features.loc[split_test_data.index]
        
        # Prepare feature matrices
        X_train = pd.concat([
            split_train_data.drop(['listing_id', 'date', 'price'], axis=1), 
            train_spatial
        ], axis=1)
        
        X_test = pd.concat([
            split_test_data.drop(['listing_id', 'date', 'price'], axis=1), 
            test_spatial
        ], axis=1)
        
        y_train = split_train_data['price']
        y_test = split_test_data['price']
        
        # Ensure X_train and X_test have the same columns
        missing_cols = set(X_train.columns) - set(X_test.columns)
        for col in missing_cols:
            X_test[col] = 0
            
        # Ensure the columns are in the same order
        X_test = X_test[X_train.columns]
        
        # Convert to PyTorch tensors
        X_train_tensor = torch.FloatTensor(X_train.values)
        y_train_tensor = torch.FloatTensor(y_train.values)
        X_test_tensor = torch.FloatTensor(X_test.values)
        
        # Initialize model for this split
        model = STRAP(
            input_dim=X_train.shape[1],
            hidden_dim=128,
            num_gru_layers=2,
            dropout=0.2
        ).to(device)
        
        # Define optimizer and loss function
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
        criterion = nn.MSELoss()
        
        # Create DataLoader for training with larger batch size and multiple workers
        train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = torch.utils.data.DataLoader(
            train_dataset, 
            batch_size=512,  # Increased batch size
            shuffle=True,
            drop_last=False,
            num_workers=4,  # Use multiple CPU cores
            pin_memory=True  # Speed up CPU to GPU transfer
        )
        
        # Train the model with early stopping
        print(f"Training on {len(X_train)} samples, testing on {len(X_test)} samples")
        
        num_epochs = 50
        patience = 7
        min_delta = 0.001
        
        best_val_loss = float('inf')
        best_model_state = None
        counter = 0
        
        # Create a validation set from training data
        train_size = int(0.8 * len(X_train_tensor))
        val_size = len(X_train_tensor) - train_size
        
        train_subset, val_subset = torch.utils.data.random_split(
            train_dataset, 
            [train_size, val_size]
        )
        
        train_subset_loader = torch.utils.data.DataLoader(
            train_subset, 
            batch_size=1024,  # Increased batch size
            shuffle=True,
            drop_last=False,
            num_workers=4,
            pin_memory=True
        )
        
        val_subset_loader = torch.utils.data.DataLoader(
            val_subset, 
            batch_size=1024,  # Increased batch size
            shuffle=False,
            drop_last=False,
            num_workers=4,
            pin_memory=True
        )
        
        # Training loop with early stopping
        for epoch in range(num_epochs):
            # Train on training subset
            model.train()
            train_loss = 0
            batch_count = 0
            
            for batch_x, batch_y in train_subset_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                
                # Forward pass
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                
                train_loss += loss.item()
                batch_count += 1
            
            if batch_count > 0:
                train_loss /= batch_count
            
            # Validate
            model.eval()
            val_loss = 0
            val_count = 0
            
            with torch.no_grad():
                for val_x, val_y in val_subset_loader:
                    val_x = val_x.to(device)
                    val_y = val_y.to(device)
                    val_outputs = model(val_x)
                    batch_loss = criterion(val_outputs, val_y)
                    val_loss += batch_loss.item()
                    val_count += 1
            
            if val_count > 0:
                val_loss /= val_count
            
            # Print progress every 5 epochs
            if (epoch + 1) % 5 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
            
            # Check for improvement
            if val_loss < best_val_loss - min_delta:
                best_val_loss = val_loss
                best_model_state = model.state_dict().copy()
                counter = 0
            else:
                counter += 1
            
            # Early stopping
            if counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        # Load best model if we found one
        if best_model_state is not None:
            model.load_state_dict(best_model_state)
        
        # Final training on all training data
        print("Final training on all training data...")
        for _ in range(5):  # A few more epochs on all data
            model.train()
            train_loss = 0
            batch_count = 0
            
            for batch_x, batch_y in train_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                
                # Forward pass
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                
                train_loss += loss.item()
                batch_count += 1
        
        # Evaluate on test data
        model.eval()
        X_test_tensor = X_test_tensor.to(device)
        with torch.no_grad():
            y_pred = model(X_test_tensor).cpu().numpy()
        
        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred) if len(set(y_test)) > 1 else np.nan
        with np.errstate(divide='ignore', invalid='ignore'):
            mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100
        
        print(f"Split {i+1} Results - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")
        
        # Generate simulated feature importance for visualization
        feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': np.random.uniform(0, 1, size=X_train.shape[1]),  # Random placeholder values
            'split': i
        })
        feature_importance_over_time.append(feature_importance)
        
        # Store results for this split
        split_results = pd.DataFrame({
            'split': i,
            'date': split_test_data['date'],
            'listing_id': split_test_data['listing_id'],
            'price': y_test,
            'predicted': y_pred,
            'error': y_test - y_pred,
            'abs_error': np.abs(y_test - y_pred),
            'pct_error': np.abs((y_test - y_pred) / (y_test + 1e-8)) * 100
        })
        
        cv_results.append(split_results)
    
    # Combine all results
    all_results = pd.concat(cv_results, ignore_index=True)
    all_feature_importance = pd.concat(feature_importance_over_time, ignore_index=True)
    
    # Calculate overall metrics
    y_true = all_results['price']
    y_pred = all_results['predicted']
    
    overall_metrics = {
        'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
        'mae': mean_absolute_error(y_true, y_pred),
        'r2': r2_score(y_true, y_pred),
        'mape': np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100,
        'explained_variance': explained_variance_score(y_true, y_pred),
        'max_error': max_error(y_true, y_pred),
        'median_absolute_error': median_absolute_error(y_true, y_pred)
    }
    
    # Calculate split-level metrics
    split_metrics = []
    for split in range(n_splits):
        split_data = all_results[all_results['split'] == split]
        if not split_data.empty:
            y_true_split = split_data['price']
            y_pred_split = split_data['predicted']
            
            split_metrics.append({
                'split': split,
                'rmse': np.sqrt(mean_squared_error(y_true_split, y_pred_split)),
                'mae': mean_absolute_error(y_true_split, y_pred_split),
                'r2': r2_score(y_true_split, y_pred_split) if len(set(y_true_split)) > 1 else np.nan,
                'mape': np.mean(np.abs((y_true_split - y_pred_split) / (y_true_split + 1e-8))) * 100,
                'n_samples': len(y_true_split)
            })
    
    split_metrics_df = pd.DataFrame(split_metrics)
    
    # Calculate daily metrics
    all_results['date_str'] = pd.to_datetime(all_results['date']).dt.strftime('%Y-%m-%d')
    
    daily_metrics = []
    for day, group in all_results.groupby('date_str'):
        y_true_day = group['price']
        y_pred_day = group['predicted']
        
        daily_metrics.append({
            'date': day,
            'rmse': np.sqrt(mean_squared_error(y_true_day, y_pred_day)),
            'mae': mean_absolute_error(y_true_day, y_pred_day),
            'r2': r2_score(y_true_day, y_pred_day) if len(set(y_true_day)) > 1 else np.nan,
            'mape': np.mean(np.abs((y_true_day - y_pred_day) / (y_true_day + 1e-8))) * 100,
            'n_samples': len(y_true_day)
        })
    
    daily_metrics_df = pd.DataFrame(daily_metrics)
    daily_metrics_df['date'] = pd.to_datetime(daily_metrics_df['date'])
    daily_metrics_df = daily_metrics_df.sort_values('date')
    
    # Error analysis
    errors = all_results['error'].values
    error_autocorr = acf(errors, nlags=7)[1:]  # Exclude lag 0
    
    # Create a results dictionary
    evaluation_results = {
        'overall_metrics': overall_metrics,
        'split_metrics': split_metrics_df,
        'daily_metrics': daily_metrics_df,
        'all_results': all_results,
        'feature_importance': all_feature_importance,
        'error_autocorrelation': error_autocorr,
        'train_listings': len(train_listing_ids),
        'test_listings': len(test_listing_ids)
    }
    
    print("\n===== Running Permutation Test for Data Leakage =====")
    permutation_test_results = []
    
    for i, (test_start, test_end) in enumerate(test_periods):
        print(f"\nRunning permutation test for Split {i+1}/{n_splits}")
        
        # Get the test data for this split
        split_test_data = all_results[all_results['split'] == i]
        
        if len(split_test_data) == 0:
            continue
            
        # Create a copy with shuffled prices
        np.random.seed(42 + i)  # Different seed for each split
        permutation_test_df = split_test_data.copy()
        
        # Get original listing IDs and true prices
        listing_ids = permutation_test_df['listing_id'].values
        original_prices = permutation_test_df['price'].values
        
        # Shuffle prices within the test set
        shuffled_prices = original_prices.copy()
        np.random.shuffle(shuffled_prices)
        permutation_test_df['shuffled_price'] = shuffled_prices
        
        # Get model predictions (these remain the same)
        predictions = permutation_test_df['predicted'].values
        
        # Calculate metrics with shuffled data
        shuffled_rmse = np.sqrt(mean_squared_error(shuffled_prices, predictions))
        shuffled_mae = mean_absolute_error(shuffled_prices, predictions)
        shuffled_r2 = r2_score(shuffled_prices, predictions) if len(set(shuffled_prices)) > 1 else np.nan
        
        # Store results
        permutation_test_results.append({
            'split': i,
            'original_rmse': np.sqrt(mean_squared_error(original_prices, predictions)),
            'original_mae': mean_absolute_error(original_prices, predictions),
            'original_r2': r2_score(original_prices, predictions) if len(set(original_prices)) > 1 else np.nan,
            'shuffled_rmse': shuffled_rmse,
            'shuffled_mae': shuffled_mae,
            'shuffled_r2': shuffled_r2,
            'n_samples': len(permutation_test_df)
        })
        
        print(f"Split {i+1} Original vs Shuffled: R² {permutation_test_results[-1]['original_r2']:.4f} vs {shuffled_r2:.4f}")
        if shuffled_r2 > 0.3:
            print("WARNING: High R² (>0.3) on shuffled data indicates potential data leakage!")
    
    # Convert to DataFrame for easier analysis
    permutation_df = pd.DataFrame(permutation_test_results)
    
    # Calculate overall shuffled metrics
    all_original_prices = all_results['price'].values
    all_predictions = all_results['predicted'].values
    
    # Shuffle all prices together
    np.random.seed(42)
    all_shuffled_prices = all_original_prices.copy()
    np.random.shuffle(all_shuffled_prices)
    
    overall_shuffled_metrics = {
        'rmse': np.sqrt(mean_squared_error(all_shuffled_prices, all_predictions)),
        'mae': mean_absolute_error(all_shuffled_prices, all_predictions),
        'r2': r2_score(all_shuffled_prices, all_predictions),
        'mape': np.mean(np.abs((all_shuffled_prices - all_predictions) / (all_shuffled_prices + 1e-8))) * 100
    }
    
    print("\n===== Overall Permutation Test Results =====")
    print(f"Original R²: {overall_metrics['r2']:.4f}")
    print(f"Shuffled R²: {overall_shuffled_metrics['r2']:.4f}")
    
    if overall_shuffled_metrics['r2'] > 0.3:
        print("\nWARNING: High R² (>0.3) on shuffled data indicates serious data leakage!")
        print("This suggests your model is using information that shouldn't be available.")
    else:
        print("\nNo significant data leakage detected in the permutation test.")
    
    # Add permutation test results to evaluation results
    evaluation_results['permutation_test'] = permutation_df
    evaluation_results['overall_shuffled_metrics'] = overall_shuffled_metrics
    
    # Create permutation test plot
    plt.figure(figsize=(10, 6))
    
    x = np.arange(len(permutation_df))
    width = 0.35
    
    plt.bar(x - width/2, permutation_df['original_r2'], width, label='Original R²')
    plt.bar(x + width/2, permutation_df['shuffled_r2'], width, label='Shuffled R²')
    
    plt.axhline(y=0.3, color='r', linestyle='--', label='Leakage Threshold (R²=0.3)')
    plt.xlabel('CV Split')
    plt.ylabel('R²')
    plt.title('Permutation Test Results: Original vs. Shuffled Prices')
    plt.xticks(x, permutation_df['split'])
    plt.legend()
    plt.tight_layout()
    
    if output_dir:
        plt.savefig(os.path.join(output_dir, 'permutation_test.png'))
        
        # Save permutation test results
        permutation_file = os.path.join(output_dir, 'permutation_test_results.csv')
        permutation_df.to_csv(permutation_file, index=False)
        print(f"Permutation test results saved to {permutation_file}")
        
        # Update summary file
        with open(os.path.join(output_dir, 'summary.txt'), 'a') as f:
            f.write("\n\nPermutation Test Results:\n")
            f.write(f"Original R²: {overall_metrics['r2']:.6f}\n")
            f.write(f"Shuffled R²: {overall_shuffled_metrics['r2']:.6f}\n")
            
            if overall_shuffled_metrics['r2'] > 0.3:
                f.write("WARNING: Potential data leakage detected!\n")
    
    plt.show()

    # Save results if output directory is provided
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        
        # Save all results
        results_file = os.path.join(output_dir, 'cv_strap_listing_split_results.csv')
        all_results.to_csv(results_file, index=False)
        print(f"Results saved to {results_file}")
        
        # Save metrics
        metrics_file = os.path.join(output_dir, 'cv_strap_listing_split_metrics.csv')
        daily_metrics_df.to_csv(metrics_file, index=False)
        print(f"Daily metrics saved to {metrics_file}")
        
        # Save feature importance
        importance_file = os.path.join(output_dir, 'cv_strap_listing_split_feature_importance.csv')
        all_feature_importance.to_csv(importance_file, index=False)
        print(f"Feature importance saved to {importance_file}")
        
        # Save model parameters
        model_info = {
            'architecture': 'STRAP (Spatio-Temporal Real estate APpraisal)',
            'input_dim': X_train.shape[1],
            'hidden_dim': 128,
            'num_gru_layers': 2,
            'dropout': 0.2
        }
        
        # Save summary
        with open(os.path.join(output_dir, 'summary.txt'), 'w') as f:
            f.write(f"Cross-Validation STRAP Neural Network with Listing ID Split\n")
            f.write(f"==========================================================\n\n")
            f.write(f"Window size: {window_size} days\n")
            f.write(f"Number of splits: {n_splits}\n")
            f.write(f"Training period: {unique_dates[0]} to {unique_dates[-1]}\n")
            f.write(f"Number of training listings: {len(train_listing_ids)}\n")
            f.write(f"Number of test listings: {len(test_listing_ids)}\n\n")
            
            f.write(f"Model Architecture:\n")
            for k, v in model_info.items():
                f.write(f"  {k}: {v}\n")
            
            f.write(f"\nOverall Metrics:\n")
            for k, v in overall_metrics.items():
                f.write(f"  {k}: {v:.6f}\n")
    
    # Print summary
    print_cv_summary(evaluation_results)
    
    # Create plots
    plot_cv_results(evaluation_results)
    
    return evaluation_results

def print_cv_summary(evaluation_results):
    """Print a summary of cross-validation results for Improved STRAP model"""
    overall = evaluation_results['overall_metrics']
    splits = evaluation_results['split_metrics']
    daily = evaluation_results['daily_metrics']
    error_autocorr = evaluation_results.get('error_autocorrelation', [0] * 7)
    
    # Print info about listing splits
    train_listings = evaluation_results.get('train_listings', 'N/A')
    test_listings = evaluation_results.get('test_listings', 'N/A')
    
    print("\n===== CROSS-VALIDATION IMPROVED STRAP MODEL WITH LISTING ID SPLIT =====")
    if train_listings != 'N/A':
        print(f"Using {train_listings} listings for training and {test_listings} listings for testing")
    
    print("\n=== Overall Metrics ===")
    print(f"RMSE: {overall['rmse']:.4f}")
    print(f"MAE: {overall['mae']:.4f}")
    print(f"R²: {overall['r2']:.4f}")
    print(f"MAPE: {overall['mape']:.4f}%")
    print(f"Explained Variance: {overall['explained_variance']:.4f}")
    print(f"Median Abs Error: {overall['median_absolute_error']:.4f}")
    
    print("\n=== Split Performance ===")
    print(splits[['split', 'rmse', 'mae', 'mape', 'r2', 'n_samples']].to_string(index=False))
    
    print("\n=== Split Statistics ===")
    print("MAE:")
    print(f"  Average: {splits['mae'].mean():.4f}")
    print(f"  Min: {splits['mae'].min():.4f} (Split {splits.loc[splits['mae'].idxmin(), 'split']})")
    print(f"  Max: {splits['mae'].max():.4f} (Split {splits.loc[splits['mae'].idxmax(), 'split']})")
    
    print("\nRMSE:")
    print(f"  Average: {splits['rmse'].mean():.4f}")
    print(f"  Min: {splits['rmse'].min():.4f} (Split {splits.loc[splits['rmse'].idxmin(), 'split']})")
    print(f"  Max: {splits['rmse'].max():.4f} (Split {splits.loc[splits['rmse'].idxmax(), 'split']})")
    
    print("\n=== Error Autocorrelation ===")
    for lag, acf_value in enumerate(error_autocorr, 1):
        print(f"  Lag {lag}: {acf_value:.4f}")
    
    print("\n=== Model Information ===")
    print("Neural network architecture: Improved STRAP (Spatio-Temporal Real estate APpraisal)")
    print("Components:")
    print("  - Property embedding layer")
    print("  - Neighborhood embedding layer")
    print("  - Actual historical temporal sequences (vs. simple repeats)")
    print("\n=== Model Information ===")
    print("Neural network architecture: Improved STRAP (Spatio-Temporal Real estate APpraisal)")
    print("Components:")
    print("  - Property embedding layer")
    print("  - Neighborhood embedding layer")
    print("  - Actual historical temporal sequences (vs. simple repeats)")
    print("  - GRU layers for temporal modeling (2 layers, 128 hidden dim)")
    print("  - 1D Convolutional layer for local feature extraction")
    print("  - Transformer encoder (2 layers, 4 heads) for global context")
    print("  - Graph convolutional layers for spatial modeling")
    print("  - Attention mechanism for focusing on relevant timesteps")
    print("  - Residual connections between components")
    print("  - Dropout (0.2) for regularization")
    print("Training:")
    print("  - Early stopping with validation set")
    print("  - Adam optimizer with weight decay")
    print("  - MSE loss function")
    print("  - Mixed training with both standard features and temporal sequences")
    print("Note: Neural networks don't provide direct feature importance like tree-based models")
    print("      Permutation-based importance estimation is computed for visualization")

def plot_cv_results(evaluation_results):
    """Plot the results from cross-validation for Improved STRAP model"""
    # Set style
    sns.set_theme(style="whitegrid")
    
    # Extract data
    daily_metrics = evaluation_results['daily_metrics']
    all_results = evaluation_results['all_results']
    splits = evaluation_results['split_metrics']
    
    # Create a figure with multiple subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Add title showing we're using listing ID split with CV for improved model
    fig.suptitle('Cross-Validation Improved STRAP Neural Network with Listing ID Split', fontsize=16)
    
    # Plot 1: Daily MAE
    sns.lineplot(
        x=pd.to_datetime(daily_metrics['date']),
        y=daily_metrics['mae'],
        marker='o',
        ax=axes[0, 0]
    )
    axes[0, 0].set_title('Mean Absolute Error by Day')
    axes[0, 0].set_xlabel('Date')
    axes[0, 0].set_ylabel('MAE')
    axes[0, 0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    axes[0, 0].xaxis.set_major_locator(mdates.MonthLocator())
    plt.setp(axes[0, 0].xaxis.get_majorticklabels(), rotation=45)
    
    # Plot 2: Cross-validation splits performance
    splits_x = splits['split']
    metrics_to_plot = ['rmse', 'mae']
    
    for metric in metrics_to_plot:
        sns.lineplot(
            x=splits_x,
            y=splits[metric],
            marker='o',
            label=metric.upper(),
            ax=axes[0, 1]
        )
    
    axes[0, 1].set_title('Performance Across CV Splits')
    axes[0, 1].set_xlabel('CV Split')
    axes[0, 1].set_ylabel('Error Metric')
    axes[0, 1].legend()
    
    # Plot 3: Actual vs Predicted (colored by split)
    scatter = axes[1, 0].scatter(
        all_results['price'],
        all_results['predicted'],
        c=all_results['split'],
        alpha=0.6,
        cmap='viridis'
    )
    min_val = min(all_results['price'].min(), all_results['predicted'].min())
    max_val = max(all_results['price'].max(), all_results['predicted'].max())
    axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'k--')
    axes[1, 0].set_title('Actual vs Predicted (Colored by CV Split)')
    axes[1, 0].set_xlabel('Actual')
    axes[1, 0].set_ylabel('Predicted')
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=axes[1, 0])
    cbar.set_label('CV Split')
    
    # Plot 4: Error distribution
    sns.histplot(all_results['error'], kde=True, ax=axes[1, 1])
    axes[1, 1].axvline(0, color='r', linestyle='--')
    axes[1, 1].set_title('Error Distribution')
    axes[1, 1].set_xlabel('Error (Actual - Predicted)')
    axes[1, 1].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.92)  # Make room for the suptitle
    plt.show()
    
    # Plot error autocorrelation
    error_acf = np.concatenate([[1], evaluation_results['error_autocorrelation']])
    lags = range(len(error_acf))
    
    plt.figure(figsize=(10, 5))
    plt.bar(lags, error_acf, alpha=0.7)
    plt.axhline(y=0, linestyle='--', color='gray')
    
    # Add confidence intervals (95%)
    conf_interval = 1.96 / np.sqrt(len(all_results['error']))
    plt.axhline(y=conf_interval, linestyle='--', color='red', alpha=0.5)
    plt.axhline(y=-conf_interval, linestyle='--', color='red', alpha=0.5)
    
    plt.xlabel('Lag')
    plt.ylabel('Autocorrelation')
    plt.title('Error Autocorrelation')
    plt.tight_layout()
    plt.show()
    
    # Additional plot: Performance by listing ID count
    # Group by listing ID and calculate average absolute error for each listing
    listing_errors = all_results.groupby('listing_id')['abs_error'].mean().reset_index()
    listing_errors = listing_errors.sort_values('abs_error')
    
    plt.figure(figsize=(12, 6))
    plt.scatter(range(len(listing_errors)), listing_errors['abs_error'], alpha=0.6)
    plt.axhline(y=listing_errors['abs_error'].mean(), color='r', linestyle='--', 
                label=f'Mean: {listing_errors["abs_error"].mean():.2f}')
    plt.title('Average Absolute Error by Listing')
    plt.xlabel('Listing Index (sorted by error)')
    plt.ylabel('Mean Absolute Error')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Feature importance visualization
    # Get top features by averaged importance
    feature_importance = evaluation_results['feature_importance']
    top_features = feature_importance.groupby('feature')['importance'].mean().nlargest(15)
    
    plt.figure(figsize=(10, 8))
    sns.barplot(
        x=top_features.values,
        y=top_features.index,
        palette='viridis'
    )
    plt.title('Top 15 Features (Estimated Importance via Permutation)')
    plt.xlabel('Relative Importance')
    plt.tight_layout()
    plt.show()
    
    # Feature importance evolution across splits
    top_5_features = feature_importance.groupby('feature')['importance'].mean().nlargest(5).index
    
    plt.figure(figsize=(12, 6))
    for feature in top_5_features:
        feature_data = feature_importance[feature_importance['feature'] == feature]
        plt.plot(
            feature_data['split'],
            feature_data['importance'],
            marker='o',
            label=feature
        )
    
    plt.title('Top 5 Feature Importance Across CV Splits')
    plt.xlabel('CV Split')
    plt.ylabel('Relative Importance')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Additional plot specifically for the improved model: Model architecture components
    plt.figure(figsize=(12, 8))
    components = [
        'Property Embedding', 
        'Neighborhood Embedding',
        'GRU Layers',
        'Conv1D Layer',
        'Transformer Encoder',
        'Graph Convolution',
        'Attention Mechanism',
        'Residual Connections'
    ]
    
    # Simulate component contributions to model performance
    np.random.seed(42)
    contributions = np.random.uniform(0.5, 1.0, size=len(components))
    contributions = contributions / contributions.sum()
    
    plt.barh(components, contributions, color=plt.cm.viridis(np.linspace(0, 1, len(components))))
    plt.title('Simulated Component Contributions in Improved STRAP Model')
    plt.xlabel('Relative Contribution')
    plt.tight_layout()
    plt.show()

# Example usage
if __name__ == "__main__":
    # Path to data files
    train_path = "train22.csv"
    train_ids_path = "train_ids.txt"
    test_ids_path = "test_ids.txt"
    
    # Output directory
    output_dir = r"C:\Users\mvk\Documents\DATA_school\thesis\Output\STRAP\Improved_CV_Model"
    
    # Run the improved model
    results = run_cv_improved_strap(
        train_path=train_path,
        train_ids_path=train_ids_path,
        test_ids_path=test_ids_path,
        output_dir=output_dir,
        window_size=5,  # Number of historical points to use in sequences
        n_splits=5,
        sample_size=None  # Set to a number like 1000 for testing, None to use all data
    )

Processing dataset: train22.csv
Loading data...
Loading train/test listing IDs...
Loaded 6291 train IDs and 1573 test IDs
Dropping simulated_price column from the dataset
Creating calculated features...
Date range in filtered data: 2023-08-07 00:00:00 to 2024-02-08 00:00:00
Number of days with data: 186
Using available data starting from 2023-08-07
No neighborhood column found, using dummy values
Pre-computing spatial features for all data...
Calculating distance to center...
Building spatial index tree...
Calculating POI density approximation...
Calculating transport access approximation...
Calculating simplified amenity score...
Processing KNN price features in chunks...
Processing chunk 1/141...
Processing chunk 21/141...
Processing chunk 41/141...
Processing chunk 61/141...
Processing chunk 81/141...
Processing chunk 101/141...
